# Supervised Learning Project: Black Friday
This project was completed for a competition held on http://datahack.analyticsvidhya.com/contest/black-friday-data-hack
# Problem Statement
The challenge was to predict purchase prices of different products purchased by customers based on historical purchase data. The dataset contains features like User_ID, Product_ID, Gender, Age, Occupation, City_Category, Stay_In_Current_City_Years, Marital_Status, Product Category (1, 2, 3), and the target/independent feature 'Purchase'. This is a regression problem.

# Exploratory Analysis
First import the important libraries for analysis

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
train = pd.read_csv('train/train.csv')
test = pd.read_csv('train/test.csv')

# Display the first record
train.head(1)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370


1. Checking the dataset for any missing values

In [5]:
train.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

Here, Product_Category_1 and Product_Category_2 have missing values. But, I'm not going to use these features in my final model, so I'm not going to fix the missing values.

2. Checking the data for outliers